In [1]:
from mbmvpa.data.loader import BIDSDataLoader

In [2]:
from mbmvpa.models.tf_mlp import MVPA_MLP

In [3]:
!ls test_example

dataset_description.json  derivatives  sub-01


In [4]:
root = './test_example/'

In [5]:
loader = BIDSDataLoader(layout=root, 
                        feature_name='zoom2',
                        task_name='mixedgamblestask',
                        process_name='SUgamble')

/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/bids/layout/models.py:152: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)
                                                 

INFO: retrieving from BIDS Layout: ...est_example/derivatives/mbmvpa | Subjects: 1 | Sessions: 0 | Runs: 1
      task-mixedgamblestask, process-SUgamble, feature-zoom2
INFO: start loading data
INFO: loaded data info. total-2
      subject_01: X(240, 119), y(240, 1)
INFO: loaded voxel mask(46, 55, 46)
INFO: loading data done


In [6]:
X,y = loader.get_data(subject_wise=False)
voxel_mask = loader.get_voxel_mask()

In [7]:
input_shape = X.shape[1]

model = MVPA_MLP(input_shape,
                 layer_dims=[1024, 1024],
                 activation="sigmoid",
                 activation_output="linear",
                 dropout_rate=0.5,
                 val_ratio=0.05,
                 optimizer="adam",
                 loss="mse",
                 learning_rate=0.001,
                 n_epoch = 80,
                 n_patience = 15,
                 n_batch = 64,
                 n_sample = 100000,
                 use_bias = True,
                 gpu_visible_devices=[2],
                 use_bipolar_balancing = False)

In [8]:
model.reset()

In [9]:
keras_model = model.model

# SHAP

In [10]:
import shap

In [11]:
import numpy as np

In [12]:
# select a set of background examples to take an expectation over
background = X[np.random.choice(X.shape[0], 100, replace=False)]



In [13]:
# explain predictions of the model on four images
e = shap.DeepExplainer(keras_model, background)


Using TensorFlow backend.
keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode. See PR #1483 for discussion.


In [14]:
# ...or pass tensors directly
# e = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), background)
shap_values = e.shap_values(X[1:5])


`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


In [18]:
X.shape

(240, 119)

# Deep LIFT

In [10]:
import deeplift
from deeplift.conversion import kerasapi_conversion as kc

In [11]:
from deeplift.layers import NonlinearMxtsMode

In [12]:
keras_model.save("test_report/example.h5",save_format='h5')

In [22]:
kc.convert_sequential_model(keras_model)

nonlinear_mxts_mode is set to: DeepLIFT_GenomicsDefault


TypeError: 'Sequential' object is not subscriptable

In [17]:
kc.convert_model_from_saved_files("test_report/example.h5")

/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/deeplift/conversion/kerasapi_conversion.py:359: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  str_data = h5py.File(h5_file).attrs["model_config"]
/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/deeplift/conversion/kerasapi_conversion.py:366: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are:

TypeError: can only concatenate str (not "KeysViewHDF5") to str

In [14]:
import h5py

In [15]:
exh5 = h5py.File("test_report/example.h5")

/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  """Entry point for launching an IPython kernel.


In [16]:
exh5.attrs.keys()

<KeysViewHDF5 ['backend', 'keras_version', 'model_config', 'training_config']>

In [23]:
exh5.attrs['keras_version']

'2.4.0'

In [37]:
dir(keras_model)

In [21]:
keras_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              122880    
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1025      
Total params: 1,173,505
Trainable params: 1,173,505
Non-trainable params: 0
_________________________________________________________________


In [18]:
exh5.attrs['model_config']

b'{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 119], "dtype": "float32", "sparse": false, "ragged": false, "name": "dense_input"}}, {"class_name": "Dense", "config": {"name": "dense", "trainable": true, "batch_input_shape": [null, 119], "dtype": "float32", "units": 1024, "activation": "sigmoid", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dropout", "config": {"name": "dropout", "trainable": true, "dtype": "float32", "rate": 0.5, "noise_shape": null, "seed": null}}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "units": 1024, "activation": "sigmoid", "use_bias": true, "kernel_initial

In [22]:
h5py.File("test_report/keras2_mnist_cnn_allconv.h5").attrs.keys()

/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  """Entry point for launching an IPython kernel.


<KeysViewHDF5 ['backend', 'keras_version', 'model_config']>

In [24]:
h5py.File("test_report/keras2_mnist_cnn_allconv.h5").attrs['keras_version']

/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  """Entry point for launching an IPython kernel.


b'2.1.6'